# Proyecto

# Libraries

In [1]:
import serial
import sys
import glob
import time
import numpy as np

# Functions

In [2]:
def listSerialPorts():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def connectToPort(n):
    ports = listSerialPorts()
    return serial.Serial(ports[n-1])

def readValueFromData(data):
    # +000.000/0 G S
    dataString = data.decode("utf-8").replace('\r\n','')
    if dataString.endswith(' G S'):
        result = dataString.replace(' G S','').replace('/','').replace('+','')
        return float(result)
    else:
        raise Exception("Invalied format")
    
def serialRead(connection):
    while True:
          try:
              connection.flushInput()
              return readValueFromData(connection.readline())
          except:
              pass
    

# Connection

In [3]:

ports = listSerialPorts()

for i in range(0,len(ports)):
    print('{0}: {1}'.format(i+1, ports[i]))

1: /dev/tty.wlan-debug
2: /dev/tty.BT-163
3: /dev/tty.Bluetooth-Incoming-Port
4: /dev/tty.usbserial-FTDCYOPE


In [4]:
port = 4

connection = connectToPort(port)

In [5]:
for i in range(0,10):
    serialRead(connection)
    time.sleep(1)

b'+000.000/0 G S\r\n'
b'+000.000/0 G S\r\n'
b'+000.000/0 G S\r\n'
b'+000.000/0 G U\r\n'
b'+000.000/0 G S\r\n'
b'+000.000/0 G S\r\n'
b'/0 G S\r\n'
b'+000.000/0 G S\r\n'
b'+000.000/0 G S\r\n'
b'+000.000/0 G S\r\n'
b'+000.000/0 G S\r\n'


In [18]:
# connection.close()

# Procedure

Data

In [6]:
scaleDivision = 0.0001 # g
minimumCharge = 0.01 # g
nominalCalibrationMass = 200 # g
massCorrection = 0 # g
EMPs = { # EMP para masa E1 según la OIML R 111
    10: 0.00002,
    20: 0.000025,
    40: 0.00005,
    60: 0.00005,
    80: 0.000075,
    100: 0.00005,
    120: 0.000075,
    140: 0.0001,
    160: 0.0001,
    180: 0.00125,
    200: 0.0001,
    220: 0.000125
}


In [8]:
import random

In [9]:
print('Iniciando procedimiento de calibración de la balanza.')
input('Presione enter para continuar.')

# PASO 1

print('|'+'-'*120+'|')

print('''
Paso 1: Prueba de excentricidad

Se debe tomar un patrón de peso de 200 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
El patrón no se debe cambiar bajo ninguna circunstancia.
      ''')

input('Presione enter para continuar.')

points = ['Central','Inferior izquierdo','Superior izquierdo','Superior derecho','Inferior derecho','Central']

eccentricityPoints = []
for i in range(0,len(points)):
    print('\nCalibración del punto {0}, favor posicionar el patrón en el punto {1}'.format(i+1,points[i]))
    keep = True
    currentPoint = []
    while keep:
        value = serialRead(connection) # 200+random.uniform(-2, 2)
        if value >= nominalCalibrationMass*0.9:
            keep = False
            currentPoint.append(value)
    print('\nFavor retire el partón de la balanza.')
    keep = True
    while keep:
        value = serialRead(connection) # 0+random.uniform(-1, 1)# 
        if value <= nominalCalibrationMass*0.1:
            keep = False
            currentPoint.append(value)
    
    eccentricityPoints.append(currentPoint)
    
eccentricityPoints = np.array(eccentricityPoints)

p0 = eccentricityPoints[-1,0]

# print(eccentricityPoints)

# print((np.vectorize(lambda row: np.abs(row[0]-row[1]-p0)))(eccentricityPoints))

# print(np.apply_along_axis(lambda row: np.abs(row[0]-row[1]-p0), 1, eccentricityPoints))

dEccentricityMax = np.max(np.apply_along_axis(lambda row: np.abs(row[0]-row[1]-p0), 1, eccentricityPoints))

print(eccentricityPoints)

print(dEccentricityMax)

# PASO 2

print('|'+'-'*120+'|')

print('''
Paso 2: Prueba por error de indicación

Se debe tomar los patrones según se van indicando en pantalla e ir posicionándolos en la pesa.
Si no se cuenta con el valor indicado o éste no es un valor estandar para las pesas de calibración, utilizar varias,
de tal forma que sumen la cantidad necesaria de masa.
Es recomendado utilizar una sola pesa para cada valor (ej. siempre utilizar la misma pesa de 200 g cuendo se necesite).
      ''')

input('Presione enter para continuar.')

indicationPoints = []
EMPkeys = list(EMPs.keys())
for i in range(0,len(EMPkeys)):
    print('\nPosicionar {0} g en la balanza.'.format(EMPkeys[i]))
    keep = True
    currentPoint = [EMPkeys[i]]
    while keep:
        value = serialRead(connection) # EMPkeys[i]+random.uniform(-2, 2)#
        if value >= EMPkeys[i]*0.98:
            keep = False
            currentPoint.append(value)
    print('\nFavor retire el partón de la balanza.')
    keep = True
    while keep:
        value = serialRead(connection) # 0+random.uniform(-1, 1)# 
        if value <= EMPkeys[i]*0.18:
            keep = False
            currentPoint.append(value)
    
    indicationPoints.append(currentPoint)

indicationPoints = np.array(indicationPoints)

# Punto de calibración (g), Indicación de la balanza (g), Indicación de descarga (g), Error de indicación (g)
indicationPoints = np.concatenate((indicationPoints,np.transpose(np.array([np.apply_along_axis(lambda row: row[1]-row[0]-EMPs[row[0]], 1, indicationPoints)]))), axis=1)

print(indicationPoints)


Iniciando procedimiento de calibración de la balanza.
|------------------------------------------------------------------------------------------------------------------------|

Paso 1: Prueba de excentricidad

Se debe tomar un patrón de peso de 200 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
El patrón no se debe cambiar bajo ninguna circunstancia.
      

Calibración del punto 1, favor posicionar el patrón en el punto Central
b'+000.000/0 G U\r\n'
b'+000.000/0 G U\r\n'
b'+200.000/0 G S\r\n'

Favor retire el partón de la balanza.
b'+000.000/0 G S\r\n'

Calibración del punto 2, favor posicionar el patrón en el punto Inferior izquierdo
b'+190.000/0 G S\r\n'

Favor retire el partón de la balanza.
b'+000.000/0 G S\r\n'

Calibración del punto 3, favor posicionar el patrón en el punto Superior izquierdo
b'+190.000/0 G S\r\n'

Favor retire el partón de la balanza.
b'+000.000/0 G S\r\n'

Calibración del punto 4, favor posicionar el patrón en el punto Superi

In [10]:
# ures y ures0, son iguales entonces se suman en una sola
ures = (scaleDivision/np.sqrt(6))*np.ones((1,len(indicationPoints)))

# Resultado de los paso 1 y 2

uexc = np.apply_along_axis(lambda row: row[1]*dEccentricityMax/(2*np.sqrt(3)*(nominalCalibrationMass+massCorrection)), 1, indicationPoints)

# Resultado del paso 3 REVISAR
urep = 0*np.ones((1,len(indicationPoints)))

# Incertidumbre de excentricidad
u2l = ures**2+uexc**2+urep**2

utrazc = np.array(list(EMPs.items()))[:,1]/np.sqrt(3/2)

uder = np.array(list(EMPs.items()))[:,1]/(3*np.sqrt(3))

# REVISAR
uempuje = 0*np.ones((1,len(indicationPoints)))

# REVISAR
uconv = 0*np.ones((1,len(indicationPoints)))

# print(utrazc)

# Incertidumbre de masa de referencia
u2m = utrazc**2+uder**2+uempuje**2+uconv**2

# Incertidumbre combinada
ue = np.sqrt(u2m + u2l)

print(u2m)



[[2.81481481e-10 4.39814815e-10 1.75925926e-09 1.75925926e-09
  3.95833333e-09 1.75925926e-09 3.95833333e-09 7.03703704e-09
  7.03703704e-09 1.09953704e-06 7.03703704e-09 1.09953704e-08]]
